# Notebook to download ACS PUMS dataset
This notebook provides the code to download the American Community Survey (ACS) Public Use Microdata Sample (PUMS) files managed by the US Census Bureau. 

In [1]:
#Import libraries
import pandas as pd
import os

from download_acs.acs_helper import ACSDataSource, adult_filter

# Link to dataset: https://www2.census.gov/programs-surveys/acs/data/pums/

In [2]:
all_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
              'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
              'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
              'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
              'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR']


In [3]:
features=['AGEP','COW','SCHL','MAR','OCCP','POBP','RELP','WKHP','SEX','RAC1P','PINCP']
features_no_relp=['AGEP','COW','SCHL','MAR','OCCP','POBP','WKHP','SEX','RAC1P','PINCP']

In [4]:
current_path = os.getcwd()
year = 2018

In [5]:
central_df = columns = ['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation', 'Place of Birth', 'Worked hours', 'Sex', 'Race', 'State', 'Income']
central_df = pd.DataFrame(columns=columns)

#Download datasets for all states
for i, state in enumerate(all_states):
    print('Start loading data for ', state)
    print(f'This is state {i + 1} of {len(all_states)}')

    state_list = [state]
    data_source = ACSDataSource(survey_year=year, horizon='1-Year', survey='person')
    ca_data = data_source.get_data(states=state_list, download=True)

    # for value in ca_data.columns:
    #     print(value)
    df = adult_filter(ca_data)

    if 'RELP' in ca_data.columns:
        df = ca_data[features]
        df.drop(['RELP'], axis=1)
    else:
        df = df[features_no_relp]

    file = 'ACS-'+str(year) +'-' + str(state_list[0]) + ".csv" 

    

    # add column with state
    df['State'] = state_list[0]

    # create column with binary income variable, below or above 50k
    df['Income'] = ['>50K' if v>50000 else '<=50K' for v in df['PINCP']]

    # rename columns to 
    df = df.rename(columns= {"AGEP": "Age", "COW": "Class of Worker", "SCHL": "Education", "MAR": "Marital Status", "OCCP":"Occupation", "POBP": "Place of Birth", "WKHP":"Worked hours", "SEX":"Sex", "RAC1P":"Race"}, errors="raise")
    
    # drop rows with NaN values
    df = df.dropna()

    # drop original income column
    df.drop('PINCP', axis=1, inplace=True)

    print('The current columns are ', df.columns)

    central_df = pd.concat([central_df, df], ignore_index=True)

    # save data as .csv file
    filepath = os.path.join(current_path, "acs_2018", file)
    df.to_csv(filepath, index=False)
    print('Len of df', len(df))
    print('Finished loading csv for this state')
    print('--------------------')

file = 'ACS-'+str(year) +'-' + 'complete' + ".csv"
filepath = os.path.join(current_path, "acs_2018", file)
central_df.to_csv(filepath, index=False)

Start loading data for  AL
This is state 1 of 51


/tmp/ipykernel_152683/329858945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = state_list[0]
/tmp/ipykernel_152683/329858945.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Income'] = ['>50K' if v>50000 else '<=50K' for v in df['PINCP']]
/tmp/ipykernel_152683/329858945.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

The current columns are  Index(['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation',
       'Place of Birth', 'RELP', 'Worked hours', 'Sex', 'Race', 'State',
       'Income'],
      dtype='object')
Len of df 22448
Finished loading csv for this state
--------------------
Start loading data for  AK
This is state 2 of 51
The current columns are  Index(['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation',
       'Place of Birth', 'RELP', 'Worked hours', 'Sex', 'Race', 'State',
       'Income'],
      dtype='object')
Len of df 3580
Finished loading csv for this state
--------------------
Start loading data for  AZ
This is state 3 of 51


/tmp/ipykernel_152683/329858945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = state_list[0]
/tmp/ipykernel_152683/329858945.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Income'] = ['>50K' if v>50000 else '<=50K' for v in df['PINCP']]


The current columns are  Index(['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation',
       'Place of Birth', 'RELP', 'Worked hours', 'Sex', 'Race', 'State',
       'Income'],
      dtype='object')
Len of df 33535
Finished loading csv for this state
--------------------
Start loading data for  AR
This is state 4 of 51


...

The current columns are  Index(['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation',
       'Place of Birth', 'RELP', 'Worked hours', 'Sex', 'Race', 'State',
       'Income'],
      dtype='object')
Len of df 33133
Finished loading csv for this state
--------------------
Start loading data for  WY
This is state 50 of 51


/tmp/ipykernel_152683/329858945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = state_list[0]
/tmp/ipykernel_152683/329858945.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Income'] = ['>50K' if v>50000 else '<=50K' for v in df['PINCP']]


The current columns are  Index(['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation',
       'Place of Birth', 'RELP', 'Worked hours', 'Sex', 'Race', 'State',
       'Income'],
      dtype='object')
Len of df 3102
Finished loading csv for this state
--------------------
Start loading data for  PR
This is state 51 of 51


/tmp/ipykernel_152683/329858945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['State'] = state_list[0]
/tmp/ipykernel_152683/329858945.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Income'] = ['>50K' if v>50000 else '<=50K' for v in df['PINCP']]


The current columns are  Index(['Age', 'Class of Worker', 'Education', 'Marital Status', 'Occupation',
       'Place of Birth', 'RELP', 'Worked hours', 'Sex', 'Race', 'State',
       'Income'],
      dtype='object')
Len of df 9103
Finished loading csv for this state
--------------------
